# Structure

The terminology I will be using is:

- Node for a cell in the gird
- The weight/distance for the time spent in a cell
- Neighbor for cells that are up, down, left, right of the cell we are referring to. 


I decided to split the logic into 2 main classes: 

1. Board, which holds the matrix of numbers and does anything that has to do with getting values from it.
2. PathFinding, the logic used to find the path in the board.

The main reason I decided to go with this split was that there was a clear distinction between the 2 pieces of logic, but also it facilitates making edits to either, as they mostly don't affect each other.

The PathFinding class holds an instance of the board on which it performs the 2 algorithms: naive and Dijkstra.

There is also an additional class for the priority queue, which is used for the Dijkstra algorithm.

Below there's an abstraction of the class structure and their functionalities, we will explore each one in more depth later on.

In [75]:
class Board:

    def print_board(self, path):
        pass

    def get_weight(self):
        pass

    def possible_moves(self, x, y):
        pass


class PathFinding:
    board: Board

    def heuristic_path(self):
        pass

    def dijkstra_path(self):
        pass


# The Board class

## Attributes

Let's look into the Board class into more details, we will start by looking at the attributes:

In [76]:
from typing import List


class Board:
    # The height and width of the board/matrix
    height: int
    width: int

    board: List[List[int]]  # The actual matrix holding the values
    moves: list  # A list of functions, which holds the functions to get the possible moves in the board from a given position
    game_mode: int = 0  # The selected game mode 

    # Variables used for printing coloured text
    GREEN = '\033[92m'
    ENDC  = '\033[0m'


## Constructor

The constructor is used to get the initial values for the width, height and game mode of the board.
In addition it initializes the moves attribute with the functions to get values (we will look into these functions later on), and it populates the board attribute with random numbers.

We can pass in different types of distribution functions with their corresponding arguments (see the random standard library for more information).

Let's look at the output of the populated board

In [2]:
import random
from typing import List

class Board:

    board: List[List[int]]

    def __init__(
        self,
        height: int,
        width: int,
        game_mode: int = 1,
        distribution = random.randint,
        **kwargs
    ) -> None:
        self.height = height
        self.width = width
        self.game_mode = game_mode
        self.moves = [
            self.move_up, 
            self.move_down, 
            self.move_left, 
            self.move_right
        ]
        self.board = []
        for _ in range(height):
            self.board.append(
                [distribution(**kwargs) for _ in range(width)]
            )

    def move_up(self):
        pass

    def move_down(self):
        pass

    def move_left(self):
        pass

    def move_right(self):
        pass


print(Board(4, 4, a = 0, b = 9).board)
print(Board(4, 4, a = 0, b = 9).board)

[[0, 8, 8, 0], [7, 2, 3, 5], [5, 7, 9, 7], [6, 6, 4, 2]]
[[6, 4, 2, 2], [4, 5, 3, 4], [5, 1, 9, 1], [2, 9, 6, 8]]


As we can see, the output is a list of size 4 containing a lists of also size 4.
Also, when creating a different instance of the board, we can see that the numbers are different from the previous, they are random each time.

## Move functions

As mentioned earlier, these functions are used to get the coordinates of a specific cell, given a current position.

For example if we are given the position (0, 0) the function move_down() will return (1, 0).
Let's look into the implementation: 

In [78]:
class Board:

    width = 4
    height = 4

    def is_within_limits(self, x, y):
        """
        Check if the given coordinates are within the board's size.
        """
        if x < self.width and x >= 0 and y < self.height and y >= 0 :
            return x, y
        
        else:
            return None

    def move_up(self, x: int, y: int):
        return self.is_within_limits(x - 1, y)

    def move_down(self, x: int, y: int):
        return self.is_within_limits(x + 1, y)
    
    def move_right(self, x: int, y: int):
        return self.is_within_limits(x, y + 1)
    
    def move_left(self, x: int, y: int):
        return self.is_within_limits(x, y - 1)


board = Board()
print('Current position (0, 0)')
print('Moving right: ' + str(board.move_right(0, 0)))
print('Moving left: ' + str(board.move_left(0, 0)))
print('Moving up: ' + str(board.move_up(0, 0)))
print('Moving down: ' + str(board.move_down(0, 0)))

Current position (0, 0)
Moving right: (0, 1)
Moving left: None
Moving up: None
Moving down: (1, 0)


With the addition of the method "is_within_limits" we can get the position of the different moves and know which ones are within the board's size. In the example above we can see that moving left and up returned None, that's because it would be impossible to move up or left from position (0, 0) as there isn't nothing above or left of the board.

These functions will save us time when we will need to gather all the possible moves from a given position. 
To gather all of these results, we can create a new method "possible_moves".

Using the attribute moves, which was initialized in the constructor, we can loop through those methods and gather the results.

In [80]:
# (Inherits itself to make use of the above functions, allowed in Jupyter notebook)
class Board(Board):

    def __init__(self):
        self.moves = [
                self.move_up, 
                self.move_down, 
                self.move_left, 
                self.move_right
            ]

    def possible_moves(self, x, y) -> list:
        return [move(x, y) for move in self.moves if move(x, y) != None]

print(Board().possible_moves(0, 0))

[(1, 0), (0, 1)]


As we can see with a single method call we are able to get all a list of the possible moves from a given coordinate.

## Get weight

This method is used to get the weight of the move, based on the given game mode.

In [ ]:
class Board(Board):

    def get_weight(self, starting_x, starting_y, x, y):
        """
        Get the weight of a node based on the selected game mode.
        """
        if self.game_mode == 0:
            return self.board[x][y]
        
        else:
            return abs(self.board[x][y] - self.board[starting_x][starting_y])

board = Board(4, 4)
print(board.board)
print('Game mode 1')
print('Weight to move to (1,0) from (0,0): ' + str(board.get_weight(0, 0, 1, 0)))
print('Weight to move to (0,1) from (0,0): ' + str(board.get_weight(0, 0, 0, 1)))
print()

board.game_mode = 0
print('Game mode 0')
print('Weight to move to (1,0) from (0,0): ' + str(board.get_weight(0, 0, 1, 0)))
print('Weight to move to (0,1) from (0,0): ' + str(board.get_weight(0, 0, 0, 1)))


[[8, 1, 4, 5], [4, 2, 1, 2], [4, 7, 2, 0], [1, 9, 8, 6]]
Game mode 1
Weight to move to (1,0) from (0,0): 4
Weight to move to (0,1) from (0,0): 7

Game mode 0
Weight to move to (1,0) from (0,0): 4
Weight to move to (0,1) from (0,0): 1


## Printing

Lastly, lets look into the printing method. This is mostly straight forward, we look through all matrix (twice, once for the rows and another time for the values) and print the results. An optional argument "path" can be passed, which will highlight elements in the positions specified.

In [ ]:
from typing import List


class Board:

    board: List[List[int]]
    GREEN = '\033[92m'
    ENDC  = '\033[0m'

    def __init__(self, height: int, width: int) -> None:
        self.height = height
        self.width = width
        self.board = []
        for _ in range(height):
            self.board.append(
                [random.randint(0, 9) for _ in range(width)]
            )

    def print_board(self, path: List[tuple] = []):
        """
        Print the board highlighting the path given.
        """
        # TODO: make space depend on the number of digits
        for x, row in enumerate(self.board):
            for y, value in enumerate(row):
                if (x, y) in path:
                    color = self.GREEN
                
                else:
                    color = ''

                print("{}%{}d".format(color, 2) % value + self.ENDC, end=" ")

            print()

print('Board without a path:')
board = Board(4, 4)
board.print_board()
print()

# Now lets give it a path
path = [(0,0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (3, 3)]
print('Let\'s look at the board with a given path:')
board.print_board(path)


Board without a path:
 3  1  3  2 
 2  0  7  7 
 8  5  5  2 
 4  5  2  0 

Let's look at the board with a given path:
 3  1  3  2 
 2  0  7  7 
 8  5  5  2 
 4  5  2  0 


# Priority queue

This is a very simple class, and it is used to store values and a "rank". When popping the queue, the item returned will be the element with the highest rank (1 being the maximum).

## Put and empty method

The put methods simply appends the given value and the rank as a tuple in the list.
The empty method returns a boolean value representing whether or not the queue is empty.

In [ ]:
from typing import List, Tuple, Any


class PriorityQueue:
    queue: List[tuple] = []

    def empty(self) -> bool:
        return len(self.queue) == 0

    def put(self, element: Tuple[int, Any]):
        self.queue.append(element)


queue = PriorityQueue()

print('The queue is empty at the beginning: ' + str(queue.empty()))

# Lets add some items
queue.put((3, 1))
queue.put((2, 5))
queue.put((5, 2))
print('The queue is not empty after adding some items: ' + str(queue.empty()))

print('The queue contains: ' + str(queue.queue))

The queue is empty at the beginning: True
The queue is not empty after adding some items: False
The queue contains: [(3, 1), (2, 5), (5, 2)]


## Get method

Things are a bit more interesting here. As we can see from the output above, the items are sorted in anyway.
Within this method we will do a lookup too find the highest ranking element and pop that off the list.

In [ ]:
from typing import List, Tuple, Any


class PriorityQueue:

    queue: List[tuple] = []

    def empty(self) -> bool:
        return len(self.queue) == 0

    def put(self, element: Tuple[int, Any]):
        self.queue.append(element)

    def get(self) -> tuple:
        if self.empty():
            raise IndexError('No elements in the list')
        
        min = 0
        for index in range(len(self.queue)):
            if self.queue[index][0] < self.queue[min][0]:
                min = index

        return self.queue.pop(min)


queue = PriorityQueue()

# Lets add the same items
queue.put((3, 1))
queue.put((2, 5))
queue.put((5, 2))

print('Initial list: ' + str(queue.queue))
print('Popping: ' + str(queue.get()))
print('Popping: ' + str(queue.get()))
print('The elements remaining are: ' + str(queue.queue))

print('Popping: ' + str(queue.get()))

try:
    queue.get()

except IndexError as error:
    print(error)

Initial list: [(3, 1), (2, 5), (5, 2)]
Popping: (2, 5)
Popping: (3, 1)
The elements remaining are: [(5, 2)]
Popping: (5, 2)
No elements in the list


# PathFinder

## Naive approach

The algorithm looks at the neighbors and makes a choice based on which path to take based on the weight/distance.


In [81]:
from typing import List

from dijkstra_algorithm.src.board import Board


class PathFinder:

    board: Board
    path: List[tuple] = [(0, 0)]

    def __init__(
        self,
        height: int,
        width: int,
        game_mode: int = 1,
        distribution = random.randint,
        **kwargs
    ) -> None:
        self.board = Board(height, width, game_mode, distribution, **kwargs)

    def print_board(self):
        self.board.print_board(self.path)

    def heuristic_path(self):
        """
        This is my approach to finding the shortest path.
        The logic can be summed up as follows:
        Check the neighbor nodes and compare their weight.
        Take the node with the smallest weight.
        If there aren't any possible moves we go back one and try again.
        Whilst doing this we keep a list of traversed nodes and the path we have taken so far.
        The process is repeated until we get to the final Node.
        """
        start_position = (0, 0)
        current_position = start_position
        final_node = (self.board.width - 1, self.board.height - 1)
        traversed_nodes = [current_position]
        path_weight = 0

        # Repeat this process until we get to the final node
        while current_position != final_node:
            smallest_weight = None
            move = None
            possible_moves = self.board.possible_moves(
                current_position[0], current_position[1]
            )

            # Go through all the possible moves from the current position.
            for x, y in possible_moves:
                weight = self.board.get_weight(*current_position, x, y)
                # If this is the final node always take it
                if (x, y) == final_node:
                    smallest_weight = weight
                    move = (x, y)
                    break

                # If the node has been traversed already ignore it
                if (x, y) in traversed_nodes:
                    continue

                # If this is the first node being traversed then, it is the best possible node for the moment
                # Compare the result of the remaining nodes with this result and get save the one with less weight.
                if smallest_weight == None:
                    smallest_weight = weight
                    move = (x, y)

                elif smallest_weight > weight:
                    smallest_weight = weight
                    move = (x, y)

            # If there were no possible moves (i.e. weight hasn't been touched)
            # Go back to the previous node resetting all the necessary variables to the previous state
            if smallest_weight == None or move == None:
                popped_node = self.path.pop()
                traversed_nodes.append(popped_node)
                path_weight -= self.board.get_weight(*popped_node, *current_position)
                current_position = self.path[-1]
                continue

            traversed_nodes.append(move)
            current_position = move
            path_weight += smallest_weight
        
        self.path = [start_position]
        self.path = traversed_nodes
        return path_weight


Let's split this up into smaller chunks.

The first step is to initialize the variables.

In [ ]:
board = Board(4, 4)

# Initialize the initial values
start_position = (0, 0)
current_position = start_position
final_node = (board.width - 1, board.height - 1)  # Our goal
traversed_nodes = [current_position]  # List of all the visited nodes
path_weight = 0

path = [start_position]  # This is a class attribute, keeps track of the chosen path so far


Now that we got these values set, we will loop the following util we reach our point.

For each loop, there's a set of variables that need to set:

In [ ]:
board = Board(4, 4)

smallest_weight = None # This keeps track of which is the smallest weight so far within the neighbors
move = None # This keeps track of which move should be made
possible_moves = board.possible_moves(
    current_position[0], current_position[1]
)  # Possible moves from the current node

print('Possible moves: ' + str(possible_moves))

Possible moves: [(1, 0), (0, 1)]


We then loop through the neighbors making some checks to identify the best candidate.

In [ ]:
board = Board(4, 4)

smallest_weight = None # This keeps track of which is the smallest weight so far within the neighbors
move = None # This keeps track of which move should be made
possible_moves = board.possible_moves(
    current_position[0], current_position[1]
)

# Go through all the possible moves from the current position.
for x, y in possible_moves:
    print('Evaluating move to: ' + str((x, y)))
    weight = board.get_weight(*current_position, x, y)
    print('Has weight: ' + str(weight))

    # If this is the final node always take it
    if (x, y) == final_node:
        print('We reached the final node')
        smallest_weight = weight
        move = (x, y)
        break

    # If the node has been traversed already ignore it
    if (x, y) in traversed_nodes:
        print('We gone through this already')
        continue

    # If this is the first node being traversed then, it is the best possible node for the moment
    # Compare the result of the remaining nodes with this result and get save the one with less weight.
    if smallest_weight == None:
        print('This is the first node being looked at, it\'s the best move at the moment')
        smallest_weight = weight
        move = (x, y)

    elif smallest_weight > weight:
        print('This move costs less, let\'s use this node instead')
        smallest_weight = weight
        move = (x, y)
    
    elif smallest_weight < weight:
        print('This move costs more, let\'s stick with our previous move')


Evaluating move to: (1, 0)
Has weight: 1
This is the first node being looked at, it's the best move at the moment
Evaluating move to: (0, 1)
Has weight: 4
This move costs more, let's stick with our previous move


After we looped through all the neighbors, if the move is still None or the weight is still None, it means that there was no path to take. In which case we go back to our previous move.

In [85]:
board = Board(4, 4)

path = [(0, 0), (1, 0)]  # an attribute of PathFinder

smallest_weight = None # This keeps track of which is the smallest weight so far within the neighbors
move = None # This keeps track of which move should be made
traversed_nodes = [(1, 1), (2, 0)]
possible_moves = board.possible_moves(
    current_position[0], current_position[1]
)

print('Current path: ' + str(path))

if smallest_weight == None or move == None:
    print('No possible moves')
    popped_node = path.pop()
    traversed_nodes.append(popped_node)
    path_weight -= board.get_weight(*popped_node, *current_position)
    current_position = path[-1]
    print('The current path is: ' + str(path))


Current path: [(0, 0), (1, 0)]
No possible moves
The current path is: [(0, 0)]


In [93]:
from dijkstra_algorithm.src.path_finder import PathFinder


path_finder = PathFinder(4, 4, 0)
path_finder.print_board()

print('Game mode 1')
print('Distance: ' + str(path_finder.heuristic_path()))
path_finder.print_board()

print('Game mode 2')
path_finder.board.game_mode = 1
print('Distance: ' + str(path_finder.heuristic_path()))
path_finder.print_board()


 8  7  1  8 
 3  0  6  6 
 9  9  7  6 
 7  6  3  4 
Game mode 1
Distance: 34
 8  7  1  8 
 3  0  6  6 
 9  9  7  6 
 7  6  3  4 
Game mode 2
Distance: 14
 8  7  1  8 
 3  0  6  6 
 9  9  7  6 
 7  6  3  4 


# Dijkstra

I used the most basic implementation of the algorithm I could come up with. There isn't anything fancy to make it more efficient.
The comments explain all the steps taken.

In [ ]:
from typing import List

from board import Board
from priority_queue import PriorityQueue


class PathFinder:

    board: Board
    path: List[tuple] = [(0, 0)]

    def __init__(self, height: int, width: int, game_mode: int = 1) -> None:
        self.board = Board(height, width, game_mode)

    def dijkstra_path(self):
        """
        Dijkstra's path implementation using a priority queue.
        """
        # For each node we will keep track of the previous node
        # This will allow us to rebuild the selected path.
        previous = {}
        start_position = (0, 0)
        # Create a dictionary with all the positions as key and distances as value
        # Set all the distances to infinity
        distances = {
            (x, y): float('inf')
            for x, row in enumerate(self.board.board)
            for y, value in enumerate(row)
        }
        # Set starting point's distance to 0
        distances[start_position] = 0

        priority_queue = PriorityQueue()
        priority_queue.put((0, start_position))

        while not priority_queue.empty():
            current_distance, current_position = priority_queue.get()

            # Compare neighbors (i.e. the possible moves from the current node)
            for neighbor in self.board.possible_moves(current_position[0], current_position[1]):
                distance = self.board.get_weight(*current_position, *neighbor) + current_distance

                # If we already have a better path to this node, then ignore current path
                if current_distance > distances[current_position]:
                    continue

                # If this is a better path, update the distances and the priority queue.                        
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    priority_queue.put((distance, neighbor))
                    previous[neighbor] = current_position

        final_node = (self.board.width - 1, self.board.height - 1)
        self.path = [start_position]
        self.shortest(final_node, previous)
        return distances[final_node]

    def shortest(self, node, previous):
        """
        Rebuild the path using the dictionary created in self.dijkstra_path
        """
        if previous.get(node):
            self.path.append(node)
            self.shortest(previous[node], previous)


There isn't much to say about this, as it's just the implementation of the algorithm. I would only add that the algorithm gets the shortest distance between the 2 points, I then use a dictionary to keep track of the path. Let's look at some results:

In [94]:
from dijkstra_algorithm.src.path_finder import PathFinder


path_finder = PathFinder(4, 4, 0)
path_finder.print_board()

print('Game mode 1')
print('Distance: ' + str(path_finder.dijkstra_path()))
path_finder.print_board()

print('Game mode 2')
path_finder.board.game_mode = 1
print('Distance: ' + str(path_finder.dijkstra_path()))
path_finder.print_board()

 7  1  4  0 
 6  9  4  7 
 4  2  2  7 
 8  7  5  8 
Game mode 1
Distance: 24
 7  1  4  0 
 6  9  4  7 
 4  2  2  7 
 8  7  5  8 
Game mode 2
Distance: 11
 7  1  4  0 
 6  9  4  7 
 4  2  2  7 
 8  7  5  8 
